<a href="https://colab.research.google.com/github/ScriptsRemote/GEE_Python/blob/main/00_Live_AmbGeo_NDVI_MENSAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Trabalhando com o Google Earth Engine*** 



1.   Importar Coleções
2.   Selecionar uma área de estudo
3.   Calculo dos índices 
4.   Construíndo Gráficos

In [ ]:
#Instalando as bibliotecas
!pip install earthengine-api
!pip install geopandas
!pip install geemap

In [ ]:
##Montando o acesso ao google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

In [ ]:
#Importando as bibliotecas
import ee
ee.Authenticate()
ee.Initialize()

In [203]:
##Importe as bibliotecas e pacotes que utilizaremos
import geopandas as gpd
import json
import pandas as pd
import geemap
import geemap.colormaps as cm
import altair as alt ##biblioteca de visualização(gráficos)
import numpy as np ## pacote de processamento de matriz (regressão linear)

In [ ]:
##Convertendo arquivo shp to json to feature
##Lendo o arquivo shp
gdf = gpd.read_file('/content/drive/MyDrive/LIVESs/ibge_municipios496_ajust.shp') 
gdf.head(5)

In [ ]:
 ##Filtrando as colunas que temos mais interesse
 roi =gdf[['NOME','geometry']]
 roi.head(5)

# ***Preparando nossa featurecollection.***

O GEE não permite a utilização direta de um arquivo .shp. Sendo assim nos iremos converter essa base para json. 

* Veja mais sobre FeatureCollection em [featurecollection](https://developers.google.com/earth-engine/guides/feature_collection_filtering)

In [ ]:
##convertendo de shp para to json
roi = roi.to_json()
##Carregando o arquivo json
roi = json.loads(roi)
##selecionando as features
roi = roi['features']
##Verificando o processo
roi

In [141]:
##Definindo filtros na feature
region = ee.FeatureCollection(roi).filter(ee.Filter.Or(ee.Filter.eq('NOME','SAO GABRIEL'), ee.Filter.eq('NOME','PORTO ALEGRE')))

In [207]:
# Defina um método para exibir blocos de imagens do Earth Engine em um folium map
Map = geemap.Map()
##Centralizando a imagem
Map = geemap.Map(location=[-30,-54], zoom_start= 8)
#Para ver uma visualização de satélite do Google como um mapa base
Map.add_basemap('HYBRID')
##Adicionando a feature  = Converte para Image (desenha o limite)
image = ee.Image().paint(region, 0, 2)
Map.addLayer(image,{'palette':'red'}, 'ROI')

display(Map)

Map(center=[-30, -54], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# ***Preparando nossa coleção de imagens.***

* Pré-processamento
* Verificando propriedades 
* Índices por mês
 

In [143]:
##Máscara de nuvens 
def maskL8sr(image):

    qaMask = image.select('QA_PIXEL').bitwise_and(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    return image.addBands(opticalBands, None, True)\
    .addBands(thermalBands, None, True).updateMask(qaMask)\
    .updateMask(saturationMask)

In [145]:
# Calculo da média mensal
# crie uma lista para os anos
years = range(2021,2022);
# crie uma lista para os meses
months = range(1,13)

def calcMonthlyMean(imageCollection):
    mylist = ee.List([])
    for y in years:
        for m in months:
            collection_month = imageCollection.filter(ee.Filter.calendarRange(y, y, 'year'))\
                                              .filter(ee.Filter.calendarRange(m, m, 'month')).median()
            ##Após definir as variáveis anuais e mensais.
            ##Devemos adicionar essa coleção a lista
            mylist = mylist.add(collection_month.set('year', y)
            .set('month', m) #Inserir propriedade mês
            .set('date', ee.Date.fromYMD(y,m,1).format('YYYY-MM-dd')) #Inserir a propriedade data
            .set('system:time_start',ee.Date.fromYMD(y,m,1))) 
    ##Fechamos a nossa função criando uma coleção de imagens
    return ee.ImageCollection.fromImages(mylist)

##Função NDVI
def NDVI (img):  
    Ndvi_image = img.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return img.addBands(Ndvi_image).clip(region).copyProperties(img, ["system:time_start"])

#Importando imagem Landsat 8 collection SR e aplicando filtros
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
                                .filterBounds(region)\
                                .filterDate('2021-01-01','2021-12-31')\
                                .map(maskL8sr)\
                                .map(NDVI)

monthlyNDVI = ee.ImageCollection(calcMonthlyMean(l8))
monthlyNDVI.first().propertyNames().getInfo() 
print(monthlyNDVI.size().getInfo())                

12


In [208]:
#NDVI reduce image
Ndvi = monthlyNDVI.select('NDVI')
##Colors map
palette = cm.palettes.ndvi
##Definindo os parâmetros barra
vis_params = {
  'min': -1,
  'max': 1,
  'palette': palette}

##Definindo os parâmetros barra
vis_params_ndvi = {'min': 0,'max': 1,'palette': palette}

##Adicionando o Layer 
Map.add_ee_layer(monthlyNDVI.median().clip(region), {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min': 0.03,'max': 0.19}, 'RGB')
Map.addLayer(Ndvi, vis_params_ndvi, 'NDVI')

##Adicionando a feature  = Converte para Image (desenha o limite)
image = ee.Image().paint(region, 0, 2)
Map.addLayer(image,{'palette':'red'}, 'ROI')
Map.add_colorbar(vis_params, label="NDVI", layer_name="NDVI")

##Finalizando a tarefa
display(Map)

Map(bottom=38809.66668701172, center=[-30.058000996853334, -53.070905279249956], controls=(WidgetControl(optio…

In [ ]:
##Função para download
download = geemap.ee_export_image_collection_to_drive(monthlyNDVI.select('NDVI'),##Lembre de selecionar a banda
                                                      region=region.geometry(), #sua área de estudo
                                                      folder='LIVE_AMBGEO', #Local onde será salvo no seu Drive
                                                      scale=30,
                                                      crs='EPSG:4674' , 
                                                      max_pixels=1e13, 
                                                      file_format='GeoTIFF') #Salvará as imagens automaticamente do Drive

Total number of images: 12

Exporting 0 ...
Exporting 1 ...
Exporting 2 ...
Exporting 3 ...
Exporting 4 ...
Exporting 5 ...
Exporting 6 ...
Exporting 7 ...
Exporting 8 ...
Exporting 9 ...
Exporting 10 ...
Exporting 11 ...


# ***Estatísticas de nossa área de estudo***

Nesta etapa vamos extrair as estatísticas por região (área de estudo) e gerar nosso dataframe para criar gráficos.

Para obter estatísticas de imagem em várias regiões armazenadas em um FeatureCollection, você pode usar image.reduceRegions() para reduzir várias regiões de uma vez. A entrada para reduceRegions() é uma Image e uma FeatureCollection. A saída é outra FeatureCollection com a saída reduceRegions() definida como propriedades em cada Feature. Neste exemplo, as médias das bandas compostas anuais do Landsat 7 em cada geometria de recurso serão adicionadas como propriedades aos recursos de entrada:

[fonte](https://developers.google.com/earth-engine/guides/reducers_reduce_regions)

<center>
<image src =https://developers.google.com/earth-engine/images/Reduce_region_diagram.png>
<center> Figure 1. An illustration of an ee.Reducer applied to an image and a region.

In [147]:
##Função para converter informações da imagem em tabela
def reduce (image):
    serie_reduce = image.reduceRegions(**{
                        'collection':region,
                        'reducer': ee.Reducer.mean().combine(**{
                        'reducer2': ee.Reducer.min(), 
                                    'sharedInputs': True}).combine(**{
                        'reducer2': ee.Reducer.max(),
                                    'sharedInputs': True}), 
                        'scale': 30
                        })
     
    serie_reduce = serie_reduce.map(lambda f: f.set({'year': image.get('year')}))\
                                .map(lambda f: f.set({'date': image.get('date')}))\
                                .map(lambda f: f.set({'month': image.get('month')}))
                                       

    return serie_reduce.copyProperties(image, ["system:time_start"])

##Aplicando a função de redução na Coleção 
data_reduce = monthlyNDVI.select('NDVI').map(reduce)\
                        .flatten()\
                        .sort('date',True)\
                        

##Verificando os dados e as propriedades
print(data_reduce.first().propertyNames().getInfo())
print(data_reduce.size().getInfo())

['system:index', 'month', 'date', 'year', 'mean', 'min', 'max', 'NOME']
24


In [148]:
##Estabelecendo a lista dos dados
Lista_df = data_reduce.reduceColumns(ee.Reducer.toList(7), ['month', 'date', 'year', 'mean', 'min', 'max', 'NOME']).values().get(0)

In [149]:
# não se esqueça que precisamos chamar o método de retorno de chamada "getInfo" para recuperar os dados
df_ndvi = pd.DataFrame(Lista_df.getInfo(), columns=['month', 'date', 'year', 'NDVImean', 'NDVImin', 'NDVImax', 'NOME'])
df_ndvi

,month,date,year,NDVImean,NDVImin,NDVImax,NOME
0,1,2021-01-01,2021,0.712591,-0.986079,0.999836,SAO GABRIEL
1,1,2021-01-01,2021,0.553182,-0.999556,0.999960,PORTO ALEGRE
2,2,2021-02-01,2021,0.779648,-0.987536,0.993244,SAO GABRIEL
3,2,2021-02-01,2021,0.563136,-0.905370,0.997919,PORTO ALEGRE
4,3,2021-03-01,2021,0.725058,-0.979553,0.998056,SAO GABRIEL
5,3,2021-03-01,2021,0.558360,-0.996759,0.999431,PORTO ALEGRE
6,4,2021-04-01,2021,0.588465,-0.995035,0.993624,SAO GABRIEL
7,4,2021-04-01,2021,0.538022,-0.945502,0.961373,PORTO ALEGRE
8,5,2021-05-01,2021,0.597391,-0.997221,0.988584,SAO GABRIEL
9,5,2021-05-01,2021,0.802264,0.476939,0.943351,PORTO ALEGRE


In [150]:
##Exportar tabela para o drive
export_precipit = df_ndvi.to_csv ('/content/drive/MyDrive/LIVESs/tabela_NDVI.csv')

In [ ]:
##Dataframe sem filtro
df_ndvi  = pd.read_csv('/content/drive/MyDrive/LIVESs/tabela_NDVI.csv')
df_ndvi

In [178]:
df_ndvi  = pd.read_csv('/content/drive/MyDrive/LIVESs/tabela_NDVI.csv')
POA = df_ndvi.loc[df_ndvi['NOME'] == 'PORTO ALEGRE']
POA

,Unnamed: 0,month,date,year,NDVImean,NDVImin,NDVImax,NOME
1,1,1,2021-01-01,2021,0.553182,-0.999556,0.999960,PORTO ALEGRE
3,3,2,2021-02-01,2021,0.563136,-0.905370,0.997919,PORTO ALEGRE
5,5,3,2021-03-01,2021,0.558360,-0.996759,0.999431,PORTO ALEGRE
7,7,4,2021-04-01,2021,0.538022,-0.945502,0.961373,PORTO ALEGRE
9,9,5,2021-05-01,2021,0.802264,0.476939,0.943351,PORTO ALEGRE
10,10,6,2021-06-01,2021,0.526905,-0.916177,0.995014,PORTO ALEGRE
12,12,7,2021-07-01,2021,0.520131,-0.991385,0.974323,PORTO ALEGRE
14,14,8,2021-08-01,2021,0.331315,-0.090098,0.891161,PORTO ALEGRE
16,16,9,2021-09-01,2021,0.415010,0.382064,0.447955,PORTO ALEGRE
18,18,10,2021-10-01,2021,0.513631,-0.996293,0.994358,PORTO ALEGRE


In [209]:
#Estatística descritiva
df_ndvi[['NDVImean','NDVImin','NDVImax']].describe()

,NDVImean,NDVImin,NDVImax
count,23.000000,23.000000,23.000000
mean,0.579304,-0.740453,0.956643
std,0.105943,0.447537,0.114171
min,0.331315,-0.999556,0.447955
25%,0.529507,-0.993210,0.957440
50%,0.563136,-0.970356,0.993624
75%,0.595413,-0.809254,0.998714
max,0.802264,0.476939,0.999960


In [210]:
##Cáculo da média dos meses
NDVI_month=df_ndvi.groupby(['month'])['NDVImin',	'NDVImean',	'NDVImax'].mean()
NDVI_month.reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,month,NDVImin,NDVImean,NDVImax
0,1,-0.992818,0.632886,0.999898
1,2,-0.946453,0.671392,0.995581
2,3,-0.988156,0.641709,0.998744
3,4,-0.970268,0.563244,0.977499
4,5,-0.260141,0.699827,0.965968
5,6,-0.916177,0.526905,0.995014
6,7,-0.939721,0.556784,0.986847
7,8,-0.189300,0.452790,0.945523
8,9,0.012988,0.542424,0.723951
9,10,-0.995835,0.550231,0.973932


In [211]:
#NDVI mensal
NDVI_month.describe()

,NDVImin,NDVImean,NDVImax
count,12.000000,12.000000,12.000000
mean,-0.747775,0.577121,0.958242
std,0.370191,0.070607,0.075629
min,-0.995835,0.452790,0.723951
25%,-0.974740,0.538544,0.965141
50%,-0.935693,0.560014,0.975715
75%,-0.706849,0.635092,0.995156
max,0.012988,0.699827,0.999898


# ***Gerando Gráficos***

Neste exemplo vamos utilizar a biblioteca Altair [click aqui e saiba mais](https://altair-viz.github.io/).

In [213]:
##Gráfico Série Temporal
alt.Chart(df_ndvi).mark_line().encode(
    x=alt.X('month', title='Data',axis=alt.Axis(labelAngle=0)),
    y=alt.Y('NDVImean:Q', title='NDVI', ),
    color=alt.Color('NOME', legend=alt.Legend(title="Municípios",orient="left"), scale=alt.Scale(scheme='dark2')),
    
    tooltip=[
             alt.Tooltip('date:N', title='Data'),
             alt.Tooltip('NDVImean:Q')
             ]).properties(title='Série histórica NDVI por ano',width=800, height=400).interactive()

alt.Chart(...)

In [214]:
##Observe também que uma dica de ferramenta foi adicionada ao gráfico; 
##passar o mouse sobre as células revela os valores das variáveis ​​selecionadas
alt.Chart(df_ndvi).mark_rect().encode(
    x='year:O',
    y='month:O',
    color=alt.Color(
        'mean(NDVImean):Q', scale=alt.Scale(scheme='redyellowgreen', domain=(-1, 1))),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('Month:O', title='Month'),
        alt.Tooltip('mean(NDVImean):Q', title='NDVI')
    ]).properties(width=600, height=300)

alt.Chart(...)

REFERENCIAS

https://altair-viz.github.io/user_guide/customization.html

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.filter.html
